<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

# Topic Modeling
## *Data Science Unit 4 Sprint 1 Assignment 4*

Analyze a corpus of Amazon reviews from Unit 4 Sprint 1 Module 1's lecture using topic modeling: 

- Fit a Gensim LDA topic model on Amazon Reviews
- Select appropriate number of topics
- Create some dope visualization of the topics
- Write a few bullets on your findings in markdown at the end
- **Note**: You don't *have* to use generators for this assignment

In [1]:
# imports 
import numpy as np
import gensim
import os
import re

from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from gensim import corpora

from gensim.models.ldamulticore import LdaMulticore

import pandas as pd

In [3]:
# Get the data from the amazon reviews into a dataframe

path = './data/Datafiniti_Amazon_Consumer_Reviews_of_Amazon_Products_May19.csv'
df = pd.read_csv(path)
df.head()

,id,dateAdded,dateUpdated,name,asins,brand,categories,primaryCategories,imageURLs,keys,...,reviews.didPurchase,reviews.doRecommend,reviews.id,reviews.numHelpful,reviews.rating,reviews.sourceURLs,reviews.text,reviews.title,reviews.username,sourceURLs
0,AVpgNzjwLJeJML43Kpxn,2015-10-30T08:59:32Z,2019-04-25T09:08:16Z,AmazonBasics AAA Performance Alkaline Batterie...,"B00QWO9P0O,B00LH3DMUO",Amazonbasics,"AA,AAA,Health,Electronics,Health & Household,C...",Health & Beauty,https://images-na.ssl-images-amazon.com/images...,"amazonbasics/hl002619,amazonbasicsaaaperforman...",...,NaN,NaN,NaN,NaN,3,https://www.amazon.com/product-reviews/B00QWO9...,I order 3 of them and one of the item is bad q...,... 3 of them and one of the item is bad quali...,Byger yang,"https://www.barcodable.com/upc/841710106442,ht..."
1,AVpgNzjwLJeJML43Kpxn,2015-10-30T08:59:32Z,2019-04-25T09:08:16Z,AmazonBasics AAA Performance Alkaline Batterie...,"B00QWO9P0O,B00LH3DMUO",Amazonbasics,"AA,AAA,Health,Electronics,Health & Household,C...",Health & Beauty,https://images-na.ssl-images-amazon.com/images...,"amazonbasics/hl002619,amazonbasicsaaaperforman...",...,NaN,NaN,NaN,NaN,4,https://www.amazon.com/product-reviews/B00QWO9...,Bulk is always the less expensive way to go fo...,... always the less expensive way to go for pr...,ByMG,"https://www.barcodable.com/upc/841710106442,ht..."
2,AVpgNzjwLJeJML43Kpxn,2015-10-30T08:59:32Z,2019-04-25T09:08:16Z,AmazonBasics AAA Performance Alkaline Batterie...,"B00QWO9P0O,B00LH3DMUO",Amazonbasics,"AA,AAA,Health,Electronics,Health & Household,C...",Health & Beauty,https://images-na.ssl-images-amazon.com/images...,"amazonbasics/hl002619,amazonbasicsaaaperforman...",...,NaN,NaN,NaN,NaN,5,https://www.amazon.com/product-reviews/B00QWO9...,Well they are not Duracell but for the price i...,... are not Duracell but for the price i am ha...,BySharon Lambert,"https://www.barcodable.com/upc/841710106442,ht..."
3,AVpgNzjwLJeJML43Kpxn,2015-10-30T08:59:32Z,2019-04-25T09:08:16Z,AmazonBasics AAA Performance Alkaline Batterie...,"B00QWO9P0O,B00LH3DMUO",Amazonbasics,"AA,AAA,Health,Electronics,Health & Household,C...",Health & Beauty,https://images-na.ssl-images-amazon.com/images...,"amazonbasics/hl002619,amazonbasicsaaaperforman...",...,NaN,NaN,NaN,NaN,5,https://www.amazon.com/product-reviews/B00QWO9...,Seem to work as well as name brand batteries a...,... as well as name brand batteries at a much ...,Bymark sexson,"https://www.barcodable.com/upc/841710106442,ht..."
4,AVpgNzjwLJeJML43Kpxn,2015-10-30T08:59:32Z,2019-04-25T09:08:16Z,AmazonBasics AAA Performance Alkaline Batterie...,"B00QWO9P0O,B00LH3DMUO",Amazonbasics,"AA,AAA,Health,Electronics,Health & Household,C...",Health & Beauty,https://images-na.ssl-images-amazon.com/images...,"amazonbasics/hl002619,amazonbasicsaaaperforman...",...,NaN,NaN,NaN,NaN,5,https://www.amazon.com/product-reviews/B00QWO9...,These batteries are very long lasting the pric...,... batteries are very long lasting the price ...,Bylinda,"https://www.barcodable.com/upc/841710106442,ht..."


In [7]:
# define stopwords

STOPWORDS = set(STOPWORDS)
len(STOPWORDS)

337

In [8]:
# define a "tokenize" function

def tokenize(text):
    return [token for token in simple_preprocess(text) if token not in STOPWORDS]

In [20]:
# figure some things out....

test = df['reviews.text']
test = list(test)
token = tokenize(test[0])
print(test[0])
token

I order 3 of them and one of the item is bad quality. Is missing backup spring so I have to put a pcs of aluminum to make the battery work.


['order',
 'item',
 'bad',
 'quality',
 'missing',
 'backup',
 'spring',
 'pcs',
 'aluminum',
 'battery',
 'work']

In [27]:
# create a list of tokenized reviews
# I think I can use list comprehension here

reviews = list(df['reviews.text'])
tokens = []

counter = 0

for review in reviews:
    token = tokenize(reviews[counter])
    tokens.append(token)
    counter = counter + 1

In [31]:
token_df = pd.DataFrame(data = {'tokens':tokens})

In [32]:
# create a dataframe of our pre-processed, tokenized reviews
token_df.head()

,tokens
0,"[order, item, bad, quality, missing, backup, s..."
1,"[bulk, expensive, way, products, like]"
2,"[duracell, price, happy]"
3,"[work, brand, batteries, better, price]"
4,"[batteries, long, lasting, price, great]"


In [33]:
# a dictionary representation of all the words in our corpus

id2word = corpora.Dictionary(token_df['tokens'])

In [37]:
# explore the dictionary

print(type(id2word))
print(id2word.token2id['duracell'])
print(len(id2word))

<class 'gensim.corpora.dictionary.Dictionary'>
16
9621


In [41]:
id2word.doc2bow(tokenize("This is a sample message I love my duracell batteries batteries batteries"))

[(16, 1), (19, 3), (93, 1), (416, 1), (3539, 1)]

In [42]:
# remove extreme values from the dataset
id2word.filter_extremes(no_below=5, no_above=.95)
len(id2word)

3581

In [43]:
# make a BOW representation of the Corpus

corpus = [id2word.doc2bow(text) for text in token_df['tokens']]

In [45]:
# what is this representing exactly? Why are there coordinates?

corpus

[[(0, 1),
  (1, 1),
  (2, 1),
  (3, 1),
  (4, 1),
  (5, 1),
  (6, 1),
  (7, 1),
  (8, 1),
  (9, 1)],
 [(10, 1), (11, 1), (12, 1), (13, 1), (14, 1)],
 [(15, 1), (16, 1), (17, 1)],
 [(9, 1), (17, 1), (18, 1), (19, 1), (20, 1)],
 [(17, 1), (18, 1), (21, 1), (22, 1), (23, 1)],
 [(18, 2),
  (20, 2),
  (24, 1),
  (25, 1),
  (26, 1),
  (27, 1),
  (28, 1),
  (29, 1),
  (30, 1),
  (31, 1),
  (32, 1),
  (33, 1),
  (34, 1),
  (35, 1),
  (36, 1),
  (37, 1),
  (38, 2),
  (39, 1),
  (40, 1)],
 [(18, 1), (41, 1), (42, 1), (43, 1), (44, 1)],
 [(18, 1), (45, 1), (46, 1), (47, 1), (48, 1), (49, 1)],
 [(12, 1),
  (15, 1),
  (17, 1),
  (50, 1),
  (51, 1),
  (52, 1),
  (53, 1),
  (54, 1),
  (55, 1)],
 [(18, 1),
  (25, 1),
  (29, 1),
  (34, 1),
  (56, 1),
  (57, 1),
  (58, 1),
  (59, 1),
  (60, 1),
  (61, 1)],
 [(17, 1),
  (18, 1),
  (20, 1),
  (21, 1),
  (24, 1),
  (62, 1),
  (63, 1),
  (64, 1),
  (65, 1),
  (66, 1),
  (67, 1),
  (68, 1),
  (69, 1),
  (70, 1),
  (71, 1)],
 [(12, 1),
  (17, 1),
  (18, 1),
 

In [47]:
lda = LdaMulticore(corpus = corpus,
                  id2word=id2word,
                  random_state = 0,
                  num_topics = 10,
                  workers = 12
                  )

In [48]:
lda.print_topics()

[(0,
  '0.046*"great" + 0.039*"tablet" + 0.026*"price" + 0.018*"use" + 0.012*"loves" + 0.012*"apps" + 0.011*"old" + 0.010*"games" + 0.010*"good" + 0.010*"easy"'),
 (1,
  '0.026*"love" + 0.023*"tablet" + 0.022*"use" + 0.021*"great" + 0.017*"like" + 0.017*"batteries" + 0.017*"amazon" + 0.016*"price" + 0.016*"easy" + 0.010*"kids"'),
 (2,
  '0.058*"batteries" + 0.034*"tablet" + 0.033*"great" + 0.026*"good" + 0.022*"price" + 0.017*"work" + 0.017*"long" + 0.014*"buy" + 0.013*"bought" + 0.010*"use"'),
 (3,
  '0.025*"love" + 0.025*"kids" + 0.020*"good" + 0.016*"games" + 0.016*"easy" + 0.015*"tablet" + 0.014*"use" + 0.013*"great" + 0.013*"nice" + 0.013*"like"'),
 (4,
  '0.023*"kindle" + 0.021*"great" + 0.020*"easy" + 0.016*"use" + 0.012*"battery" + 0.011*"amazon" + 0.011*"product" + 0.010*"love" + 0.010*"bought" + 0.010*"screen"'),
 (5,
  '0.034*"tablet" + 0.030*"great" + 0.027*"use" + 0.020*"amazon" + 0.018*"bought" + 0.017*"price" + 0.012*"batteries" + 0.012*"love" + 0.011*"easy" + 0.011*"kid

In [50]:
# visualize the different topics

import pyLDAvis.gensim

pyLDAvis.enable_notebook()

In [51]:
pyLDAvis.gensim.prepare(lda, corpus, id2word)

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2      0.045927 -0.038100       1        1  18.135023
3     -0.050437  0.029851       2        1  11.501458
7      0.046959  0.000440       3        1  10.901688
5     -0.013402 -0.011267       4        1   9.976954
6     -0.010728 -0.026955       5        1   9.149192
1      0.006124  0.003656       6        1   8.637388
0     -0.045480 -0.023676       7        1   8.594474
8      0.042204  0.025423       8        1   7.803006
4      0.003593  0.057431       9        1   7.688014
9     -0.024760 -0.016803      10        1   7.612803, topic_info=           Term         Freq        Total Category  logprob  loglift
21        great  8936.000000  8936.000000  Default  30.0000  30.0000
18    batteries  6687.000000  6687.000000  Default  29.0000  29.0000
2075     kindle  3122.000000  3122.000000  Default  28.0000  28.0000
2432     tablet  7217.000000  7217.000000  Default  27.0000  27.0000
90         love  3624.000000  3624.000000  Default  26.0000  26.0000
...         ...          ...          ...      ...      ...      ...
9          work   142.381653  2060.242188  Topic10  -5.1025  -0.0967
29          buy   140.436340  2184.182373  Topic10  -5.1163  -0.1689
28       bought   150.093979  3246.260742  Topic10  -5.0498  -0.4987
105        year   130.463287  1882.301025  Topic10  -5.1899  -0.0938
18    batteries   150.951157  6687.608887  Topic10  -5.0441  -1.2157

[807 rows x 6 columns], token_table=      Topic      Freq  Term
term                       
56        1  0.444078    aa
56        2  0.024004    aa
56        3  0.160828    aa
56        4  0.098417    aa
56        5  0.043208    aa
...     ...       ...   ...
105       9  0.033470  year
105      10  0.069064  year
1374      1  0.263228   yep
1374      4  0.131614   yep
1374      8  0.394841   yep

[3223 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[3, 4, 8, 6, 7, 2, 1, 9, 5, 10])

In [52]:
# how many topics should we be using?

from gensim.models.coherencemodel import CoherenceModel

def compute_coherence_values(dictionary, corpus, limit, start=2, step=3, passes=5):
    """
    Compute c_v coherence for various number of topics
    
    parameters:
    ----------
    dictionary : gensim dictionary
    corpus : gensim corpus
    limit : max number of topics
    passes : the number of times the entire lda model & coherence values are calculated
    
    returns:
    --------
    coherence_values : coherence values corresponding to the LDA model with respective number of topics
    """
    
    coherence_values = []
    
    for iter_ in range(passes):
        for num_topics in range(start, limit, step):
            model = LdaMulticore(corpus=corpus, num_topics=num_topics, id2word=dictionary, workers=4)
            coherencemodel = CoherenceModel(model=model, dictionary=dictionary, corpus=corpus, coherence='u_mass')
            coherence_values.append({'pass': iter_,
                                    'num_topics': num_topics,
                                    'coherence_score': coherencemodel.get_coherence()
                                    })
    return coherence_values

In [ ]:
coherence_values = compute_coherence_values(dictionary=id2word,
                                               corpus = corpus,
                                               start = 2,
                                               limit = 40,
                                               step = 2,
                                               passes = 100)

## Stretch Goals

* Incorporate Named Entity Recognition in your analysis
* Incorporate some custom pre-processing from our previous lessons (like spacy lemmatization)
* Analyze a dataset of interest to you with topic modeling